## Input: drug_name, Output: Rxnorm

In [3]:
from biothings_client import get_client

# get mydrug client
md = get_client('drug')

def names2rxnorm(drug_name_list):
    results = {}
    for _drug in drug_name_list:
        query_term = 'aeolus.drug_name:' + _drug
        _result = md.query(query_term, fields='aeolus.drug_code')
        if _result['hits'] != []:
            results[_drug] = [_record['aeolus']['drug_code'] for _record in _result['hits']]
        else:
            results[_drug] = ['no results found']
    return results
    

### Example

In [4]:
names2rxnorm(['varenicline', 'Tranylcypromine', 'topiramate'])

{'Methandriol': ['6817'], 'topiramate': ['38404'], 'varenicline': ['591622']}

## Input: Rxnorm, Output: Enzyme Uniprot ID & Gene Symbol

In [13]:
def rxnorms2enzyme(rxnorm_list):
    results = {}
    for _rxnorm in rxnorm_list:
        query_term = 'aeolus.drug_code:' + _rxnorm
        _result = md.query(query_term, fields='drugbank.enzymes.name, drugbank.enzymes.uniprot')
        if _result['hits'] != []:
            results[_rxnorm] = [_record['drugbank'] for _record in _result['hits']]
        else:
            results[_rxnorm] = ['no results found']
    return results

### Example

In [16]:
rxnorms2enzyme(['38404', '3418'])

{'3418': [{'enzymes': {'name': 'Cytochrome P450 3A4', 'uniprot': 'P08684'}}],
 '38404': [{'enzymes': [{'name': 'Cytochrome P450 2C19', 'uniprot': 'P33261'},
    {'name': 'Cytochrome P450 3A4', 'uniprot': 'P08684'}]}]}

## Input: two drug_names, Output: whether having common enzymes

In [71]:
def rxnorms2enzymes(rxnorm_list):
    results = {}
    for _rxnorm in rxnorm_list:
        query_term = 'aeolus.drug_code:' + _rxnorm
        _result = md.query(query_term, fields='drugbank.enzymes.uniprot')
        if 'hits' in _result:
            results[_rxnorm] = [_record['uniprot'] for _record in _result['hits'][0]['drugbank']['enzymes']]
        else:
            results[_rxnorm] = ['no results found']
    return results
            

In [72]:
def compareenzymes(drug_name_list):
    results = rxnorms2enzymes(drug_name_list)
    uniprot_list = []
    for drug, uniprot in results.items():
        uniprot_list.append(uniprot)
    overlap = list(set(uniprot_list[0]) & set(uniprot_list[1]))
    if overlap:
        return {'overlap': overlap}
    else:
        print('no overlap')

### Example

In [74]:
rxnorms2enzymes(['36567', '38404'])

{'36567': ['P08684',
  'P20815',
  'P24462',
  'P10632',
  'P11712',
  'P33261',
  'P10635',
  'P20813',
  'P22309',
  'P35503',
  'P16662'],
 '38404': ['P33261', 'P08684']}

In [76]:
compareenzymes(['36567', '38404'])

{'overlap': ['P08684', 'P33261']}

In [77]:
mg = get_client('gene')
def uniprot2symbol(uniprot_id_list):
    results = [['uniprot_id', 'gene_name', 'gene_symbol']]
    for _uniprot in uniprot_id_list:
        query_term = 'uniprot.Swiss-Prot:' + _uniprot
        _result = mg.query(query_term)
        if _result['hits'] != []:
            results.append([_uniprot, _result['hits'][0]['name'], _result['hits'][0]['symbol']])
    return results

In [79]:
uniprot2symbol(compareenzymes(['36567', '38404'])['overlap'])

[['uniprot_id', 'gene_name', 'gene_symbol'],
 ['P08684', 'cytochrome P450 family 3 subfamily A member 4', 'CYP3A4'],
 ['P33261', 'cytochrome P450 family 2 subfamily C member 19', 'CYP2C19']]

In [80]:
import plotly.figure_factory as ff

In [84]:
table = ff.create_table(uniprot2symbol(compareenzymes(['36567', '38404'])['overlap']))

In [85]:
import plotly.plotly as py
import plotly
py.iplot(table, filename='simple_table')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~kevinxin90/0 or inside your plot.ly account where it is named 'simple_table'
